### Cab-Driver Agent

In [1]:
# Importing libraries
import numpy as np
import random
import math
from collections import deque
import collections
import pickle

# for building DQN model
from keras import layers
from keras import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam

# for plotting graphs
import matplotlib.pyplot as plt

# Import the environment
from Env import CabDriver

#### Defining Time Matrix

In [2]:
# Loading the time matrix provided
Time_matrix = np.load("TM.npy")

In [3]:
# Checking characterstics of Time_Matrix
print(type(Time_matrix))
print(Time_matrix.max())
print(Time_matrix.min())
print(Time_matrix.mean())
print(Time_matrix.var())

<class 'numpy.ndarray'>
11.0
0.0
3.0542857142857143
7.93705306122449


#### Tracking the state-action pairs for checking convergence


In [4]:
#Defining a function to save the Q-dictionary as a pickle file
def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

### Agent Class

If you are using this framework, you need to fill the following to complete the following code block:
1. State and Action Size
2. Hyperparameters
3. Create a neural-network model in function 'build_model()'
4. Define epsilon-greedy strategy in function 'get_action()'
5. Complete the function 'append_sample()'. This function appends the recent experience tuple <state, action, reward, new-state> to the memory
6. Complete the 'train_model()' function with following logic:
   - If the memory size is greater than mini-batch size, you randomly sample experiences from memory as per the mini-batch size and do the following:
      - Initialise your input and output batch for training the model
      - Calculate the target Q value for each sample: reward + gamma*max(Q(s'a,))
      - Get Q(s', a) values from the last trained model
      - Update the input batch as your encoded state-action and output batch as your Q-values
      - Then fit your DQN model using the updated input and output batch.

In [5]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        # Define size of state and action
        self.state_size = state_size
        self.action_size = action_size

        # Write here: Specify you hyper parameters for the DQN
        self.discount_factor = 0.9
        self.learning_rate = 0.01
        self.epsilon_max = 1
        self.epsilon_decay = 0.0009
        self.epsilon_min = 0
        self.batch_size = 32        
        # create replay memory using deque
        self.memory = deque(maxlen=2000)

        # create main model and target model
        self.model = self.build_model()
        self.env1 = CabDriver()

    # approximate Q function using Neural Network
    def build_model(self):
        
        model = Sequential()
        # Write your code here: Add layers to your neural nets       
        model.add(Dense(32, input_dim=self.state_size, activation='relu', kernel_initializer='he_uniform'))
        model.add(Dense(32, activation='relu', kernel_initializer='he_uniform'))
        model.add(Dense(32, activation='relu', kernel_initializer='he_uniform'))
        # the output layer: output is of size num_actions
        model.add(Dense(self.action_size, activation='relu', kernel_initializer='he_uniform'))
        model.compile(loss='mse',optimizer=Adam(lr=self.learning_rate))
        model.summary
        return model



    def get_action(self, state,episode):
        poss_index, actions = self.env1.requests(state) 
        epsilon = (self.epsilon_min + (self.epsilon_max - self.epsilon_min) * np.exp(-self.epsilon_decay*episode))
        if np.random.rand() <= epsilon:
            return random.choice(actions)
        else:
            encoded_state =np.array(env.state_encod_arch1(state)).reshape(1,36)
            q_value = self.model.predict(encoded_state)    
            allowed_q_values = [q_value[0][i] for i in poss_index]
            action_index = possible_actions_index[np.argmax(allowed_q_values)]
            return self.env1.action_space[action_index]
        
    

    def append_sample(self, state, action, reward, next_state,done):
        # Write your code here:
        # save sample <s,a,r,s'> to the replay memory
        self.memory.append((state, action, reward, next_state, done))

    # pick samples randomly from replay memory (with batch_size) and train the network
    def train_model(self):
         if len(self.memory) > self.batch_size:
            # Sample batch from the memory
            mini_batch = random.sample(self.memory, self.batch_size)
            update_output = np.zeros((self.batch_size, self.state_size))
            update_input = np.zeros((self.batch_size, self.state_size))
            actions, rewards, done = [], [], []
            
            for i in range(self.batch_size):
                state, action, reward, next_state, terminal_state = mini_batch[i]
                actions = self.env1.action_space
                actions.append(actions.index(tuple(action)))
                rewards.append(reward)
                done.append(terminal_state)
                
                update_input[i] = env.state_encod_arch1(state)
                update_output[i] = env.state_encod_arch1(next_state)
                
            # Write your code from here
            # 1. Predict the target from earlier model
            target = self.model.predict(update_input)   

            # 2. Get the target for the Q-network
            target_qval = self.model.predict(update_output)

            #3. Update your 'update_output' and 'update_input' batch
            for i in range(self.batch_size):
                if done[i]:
                    target[i][actions[i]] = rewards[i]
                else: # non-terminal state
                    target[i][actions[i]] = rewards[i] + self.discount_factor * np.max(target_qval[i])  
                
                
            # 4. Fit your model and track the loss values
            self.model.fit(update_input, target, batch_size=self.batch_size, epochs=1, verbose=0)
                
       



    def save(self, name):
        self.model.save_weights(name)

In [6]:
Episodes = 15000

#  environment
env = CabDriver()

# get size of state and action from environment
state_size = 36
action_size = len(env.action_space)  
print(state_size)
rewards, timesteps, episodes = [], [], []
agent = DQNAgent(state_size,action_size)
write_threshold =1000

36


D:\Anacoda\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


### DQN block

In [7]:
for episode in range(Episodes):

    # Write code here
    # Call the environment
    # Call all the initialised variables of the environment
    terminal_state = False
    score = 0
    state =  env.reset()
    #Call the DQN agent
    
    count = 0
    while not terminal_state:
        count += 1
         # 1. Pick epsilon-greedy action from possible actions for the current state
        action = agent.get_action(state,episode)
        # 2. Evaluate your reward and next state
        reward = env.reward_func(state,action,Time_matrix)
        next_state , terminal_state = env.next_state_func(state,action,Time_matrix)
        # 3. Append the experience to the memory
        agent.append_sample(state,action,reward,next_state,terminal_state)
        score = score + reward
         # 4. Train the model by calling function agent.train_model
        if count%20 == 0:
            agent.train_model()
       
        state= next_state
        
       
        # 5. Keep a track of rewards, Q-values, loss
    rewards.append(score)
    episodes.append(episode)
    if episode%write_threshold == 0:
        # Saving the models
        agent.save("model_weights.h5")
        # Saving the rewards per episode as a pickle file
        print(episode, "Done")
        save_pickle(rewards,"rewards_per_episode")
        

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

### Tracking Convergence

#### Epsilon-decay sample function

<div class="alert alert-block alert-info">
Try building a similar epsilon-decay function for your model.
</div>

In [ ]:
time = np.arange(0,10000)
epsilon = []
for i in range(0,10000):
    epsilon.append(0 + (1 - 0) * np.exp(-0.0009*i))

In [ ]:
plt.plot(time, epsilon)
plt.show()